In [1]:
import json
from tqdm.auto import tqdm
import torch

from transformers import BertTokenizer, AutoTokenizer
from data_utils import get_examples_from_dialogues, convert_state_dict, load_dataset
from data_utils import OntologyDSTFeature, DSTPreprocessor, _truncate_seq_pair

from torch.cuda.amp import autocast,  GradScaler 

In [2]:
torch.__version__
use_amp = True

In [3]:
train_data_file = "/opt/ml/input/data/train_dataset/train_dials.json"
slot_meta = json.load(open("/opt/ml/input/data/train_dataset/slot_meta.json"))
ontology = json.load(open("/opt/ml/input/data/train_dataset/ontology.json"))
train_data, dev_data, dev_labels = load_dataset(train_data_file)

In [4]:
from argparse import Namespace

args = {
    'hidden_dim': 300,
    'num_rnn_layers': 1,
    'zero_init_rnn': False,
    'max_seq_length': 64,
    'max_label_length': 12,
    'attn_head': 4,
    'fix_utterance_encoder': False,
    'task_name': 'sumbtgru',
    'distance_metric': 'euclidean',
    'model_name_or_path': 'monologg/koelectra-base-v3-discriminator',
    'warmup_ratio': 0.1,
    'learning_rate': 5e-5,
    'weight_decay': 0.01,
    'num_train_epochs': 10,
}



args = Namespace(**args)
args.model_name_or_path = 'dsksd/bert-ko-small-minimal'

In [5]:
train_examples = get_examples_from_dialogues(data=train_data,
                                             user_first=True,
                                             dialogue_level=True)

dev_examples = get_examples_from_dialogues(data=dev_data,
                                           user_first=True,
                                           dialogue_level=True)

In [6]:
train_examples[0][5]

DSTInputExample(guid='snowy-hat-8324:관광_식당_11-5', context_turns=['', '서울 중앙에 있는 박물관을 찾아주세요', '안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가를 받고 있습니다.', '좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?', '전화번호는 983880764입니다. 더 필요하신 게 있으실까요?', '네 관광지와 같은 지역의 한식당을 가고싶은데요 야외석이 있어야되요', '생각하고 계신 가격대가 있으신가요?', '음.. 저렴한 가격대에 있나요?', '죄송하지만 저렴한 가격대에는 없으시네요.', '그럼 비싼 가격대로 다시 찾아주세요'], current_turn=['좋습니당 토요일 18:00에 1명 예약가능한가요?', '외계인의맛집은 어떠신가요? 대표 메뉴는 한정식입니다.'], label=['관광-종류-박물관', '관광-지역-서울 중앙', '관광-이름-문화역서울 284', '식당-가격대-비싼', '식당-지역-서울 중앙', '식당-종류-한식당', '식당-야외석 유무-yes', '식당-예약 요일-토요일', '식당-예약 시간-18:00', '식당-예약 명수-1', '식당-이름-외계인의맛집'])

In [7]:
len(train_data[0]['dialogue'])

16

In [8]:
max_turn = max([len(e['dialogue']) for e in train_data])
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

In [9]:
from transformers import BertTokenizer
from data_utils import get_examples_from_dialogues, convert_state_dict, load_dataset
from data_utils import OntologyDSTFeature, DSTPreprocessor, _truncate_seq_pair

class SUMBTPreprocessor(DSTPreprocessor):
    def __init__(
        self,
        slot_meta,
        src_tokenizer,
        trg_tokenizer=None,
        ontology=None,
        max_seq_length=64,
        max_turn_length=14,
    ):
        self.slot_meta = slot_meta
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer if trg_tokenizer else src_tokenizer
        self.ontology = ontology
        self.max_seq_length = max_seq_length
        self.max_turn_length = max_turn_length

    def _convert_example_to_feature(self, example):
        guid = example[0].guid.rsplit("-", 1)[0]  # dialogue_idx
        turns = []
        token_types = []
        labels = []
        num_turn = None
        for turn in example[: self.max_turn_length]:
            assert len(turn.current_turn) == 2
            uttrs = []
            for segment_idx, uttr in enumerate(turn.current_turn):
                token = self.src_tokenizer.encode(uttr, add_special_tokens=False)
                uttrs.append(token)

            _truncate_seq_pair(uttrs[0], uttrs[1], self.max_seq_length - 3)
            tokens = (
                [self.src_tokenizer.cls_token_id]
                + uttrs[0]
                + [self.src_tokenizer.sep_token_id]
                + uttrs[1]
                + [self.src_tokenizer.sep_token_id]
            )
            token_type = [0] * (len(uttrs[0]) + 2) + [1] * (len(uttrs[1]) + 1)
            if len(tokens) < self.max_seq_length:
                gap = self.max_seq_length - len(tokens)
                tokens.extend([self.src_tokenizer.pad_token_id] * gap)
                token_type.extend([0] * gap)
            turns.append(tokens)
            token_types.append(token_type)
            label = []
            if turn.label:
                slot_dict = convert_state_dict(turn.label)
            else:
                slot_dict = {}
            for slot_type in self.slot_meta:
                value = slot_dict.get(slot_type, "none")
                
                if value in ontology[slot_type]:
                    label_idx = ontology[slot_type].index(value)
                else:
                    label_idx = ontology[slot_type].index('none')
               
                label.append(label_idx)
            labels.append(label)
        num_turn = len(turns)
        if len(turns) < self.max_turn_length:
            gap = self.max_turn_length - len(turns)
            for _ in range(gap):
                dummy_turn = [self.src_tokenizer.pad_token_id] * self.max_seq_length
                turns.append(dummy_turn)
                token_types.append(dummy_turn)
                dummy_label = [-1] * len(self.slot_meta)
                labels.append(dummy_label)
        return OntologyDSTFeature(
            guid=guid,
            input_ids=turns,
            segment_ids=token_types,
            num_turn=num_turn,
            target_ids=labels,
        )

    def convert_examples_to_features(self, examples):
        return list(map(self._convert_example_to_feature, tqdm(examples)))

    def recover_state(self, pred_slots, num_turn):
        states = []
        
        for pred_slot in pred_slots[:num_turn]:
            state = []
            for s, p in zip(self.slot_meta, pred_slot):
                v = self.ontology[s][p]
                if v != 'none':
                    state.append(f'{s}-{v}')
                    
            states.append(state)
            
        return states

    def collate_fn(self, batch):
        guids = [b.guid for b in batch]
        input_ids = torch.LongTensor([b.input_ids for b in batch])
        segment_ids = torch.LongTensor([b.segment_ids for b in batch])
        input_masks = input_ids.ne(self.src_tokenizer.pad_token_id)
        target_ids = torch.LongTensor([b.target_ids for b in batch])
        num_turns = [b.num_turn for b in batch]
        return input_ids, segment_ids, input_masks, target_ids, num_turns, guids

In [10]:
len(train_examples)

6301

In [11]:
processor = SUMBTPreprocessor(slot_meta,
                              tokenizer,
                              ontology=ontology,  # predefined ontology
                              max_seq_length=64,  # 각 turn마다 최대 길이
                              max_turn_length=max_turn)  # 각 dialogue의 최대 turn 길이
train_features = processor.convert_examples_to_features(train_examples)
dev_features = processor.convert_examples_to_features(dev_examples)

In [12]:
print(len(train_features))  # 대화 level의 features
print(len(dev_features))

6301
699


In [13]:
# input_ids, segment_ids, input_masks, target_ids, num_turns, guids 
# print()
# model = None

In [14]:
# a = AutoForUtteranceEncoding.from_pretrained(args.model_name_or_path).to(device)

In [15]:
# b.hidden_states[-1].shape, input_ids.view(-1, 64).shape

In [16]:
# b = a.base(input_ids.view(-1, 64), segment_ids.view(-1, 64), input_masks.long().view(-1, 64),
# #             output_attentions=False,
#             output_hidden_states=True,
#             return_dict=True,
#         )
# print()

In [17]:
# b[0].shape

In [18]:
# a.base(input_ids=input_ids.view(-1, 64), token_type_ids=segment_ids.view(-1, 64), attention_mask=input_masks.view(-1, 64))[0].shape

In [19]:
# c = AutoForUtteranceEncoding.from_pretrained('dsksd/bert-ko-small-minimal').to(device)

In [20]:
# d = c.base(input_ids.view(-1, 64), segment_ids.view(-1, 64), input_masks.long().view(-1, 64),
# #             output_attentions=False,
#             output_hidden_states=True,
#             return_dict=True,
#         )

In [21]:
# d.last_hidden_state.shape

In [22]:
"""
Most of code is from https://github.com/SKTBrain/SUMBT
"""

import math
import os.path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CosineEmbeddingLoss, CrossEntropyLoss
from transformers import BertModel, BertPreTrainedModel, AutoModel, AutoModelForPreTraining, AutoConfig

class AutoForUtteranceEncoding(nn.Module):
    def __init__(self, config, model):
        super().__init__()
        self.config = config
        self.base = model

    def forward(self, input_ids, token_type_ids, attention_mask):
        return self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=False,
            output_hidden_states=True,
            return_dict=True,
        )
    
    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs):
        config = AutoConfig.from_pretrained(pretrained_model_name_or_path, **kwargs)
        model = AutoModelForPreTraining.from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)
        ret = cls(config, model)
        return ret

class BertForUtteranceEncoding(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForUtteranceEncoding, self).__init__(config)

        self.config = config
        self.base = BertModel(config)

    def forward(self, input_ids, token_type_ids, attention_mask):
        return self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=False,
            output_hidden_states=True,
            return_dict=True,
        )


class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

        self.scores = None

    def attention(self, q, k, v, d_k, mask=None, dropout=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            mask = mask.unsqueeze(1)
            mask = mask.to(dtype=scores.dtype)
            mask = (1.0 - mask) * -10000.0
            scores = scores + mask
            
        scores = F.softmax(scores, dim=-1)

        if dropout is not None:
            scores = dropout(scores)

        self.scores = scores
        output = torch.matmul(scores, v)
        return output

    def forward(self, q, k, v, mask=None):
        bs = q.size(0)

        # perform linear operation and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # transpose to get dimensions bs * h * sl * d_model
        k = k.transpose(1, 2)
        q = q.transpose(1, 2)
        v = v.transpose(1, 2)

        scores = self.attention(q, k, v, self.d_k, mask, self.dropout)

        # concatenate heads and put through final linear layer
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
        output = self.out(concat)
        return output

    def get_scores(self):
        return self.scores


class SUMBT(nn.Module):
    def __init__(self, args, num_labels, device):
        super(SUMBT, self).__init__()

        self.hidden_dim = args.hidden_dim
        self.rnn_num_layers = args.num_rnn_layers
        self.zero_init_rnn = args.zero_init_rnn
        self.max_seq_length = args.max_seq_length
        self.max_label_length = args.max_label_length
        self.num_labels = num_labels
        self.num_slots = len(num_labels)
        self.attn_head = args.attn_head
        self.device = device

        ### Utterance Encoder
        self.utterance_encoder = AutoForUtteranceEncoding.from_pretrained(
            args.model_name_or_path
        )
        self.bert_output_dim = self.utterance_encoder.config.hidden_size
        self.hidden_dropout_prob = self.utterance_encoder.config.hidden_dropout_prob
        if args.fix_utterance_encoder:
            for p in self.utterance_encoder.base.pooler.parameters():
                p.requires_grad = False

        ### slot, slot-value Encoder (not trainable)
        self.sv_encoder = AutoForUtteranceEncoding.from_pretrained(
            args.model_name_or_path
        )
        # os.path.join(args.bert_dir, 'bert-base-uncased.model'))
        for p in self.sv_encoder.base.parameters():
            p.requires_grad = False

        self.slot_lookup = nn.Embedding(self.num_slots, self.bert_output_dim)
        self.value_lookup = nn.ModuleList(
            [nn.Embedding(num_label, self.bert_output_dim) for num_label in num_labels]
        )

        ### Attention layer
        self.attn = MultiHeadAttention(self.attn_head, self.bert_output_dim, dropout=0)

        ### RNN Belief Tracker
        self.nbt = nn.GRU(
            input_size=self.bert_output_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.rnn_num_layers,
            dropout=self.hidden_dropout_prob,
            batch_first=True,
        )
        self.init_parameter(self.nbt)

        if not self.zero_init_rnn:
            self.rnn_init_linear = nn.Sequential(
                nn.Linear(self.bert_output_dim, self.hidden_dim),
                nn.ReLU(),
                nn.Dropout(self.hidden_dropout_prob),
            )

        self.linear = nn.Linear(self.hidden_dim, self.bert_output_dim)
        self.layer_norm = nn.LayerNorm(self.bert_output_dim)

        ### Measure
        self.metric = torch.nn.PairwiseDistance(p=2.0, eps=1e-06, keepdim=False)

        ### Classifier
        self.nll = CrossEntropyLoss(ignore_index=-1)

        ### Etc.
        self.dropout = nn.Dropout(self.hidden_dropout_prob)

    def initialize_slot_value_lookup(self, label_ids, slot_ids):

        self.sv_encoder.eval()

        # Slot encoding
        slot_type_ids = torch.zeros(slot_ids.size(), dtype=torch.long).to(
            slot_ids.device
        )
        slot_mask = slot_ids > 0
        hid_slot = self.sv_encoder(
            slot_ids.view(-1, self.max_label_length),
            slot_type_ids.view(-1, self.max_label_length),
            slot_mask.view(-1, self.max_label_length),
        ).hidden_states[-1]
        
        hid_slot = hid_slot[:, 0, :]
        hid_slot = hid_slot.detach()
    
        self.slot_lookup = nn.Embedding.from_pretrained(hid_slot, freeze=True)
        assert self.slot_lookup.weight.shape == (self.num_slots, self.bert_output_dim)

        for s, label_id in enumerate(label_ids):
            label_type_ids = torch.zeros(label_id.size(), dtype=torch.long).to(
                label_id.device
            )
            label_mask = label_id > 0
            hid_label = self.sv_encoder(
                label_id.view(-1, self.max_label_length),
                label_type_ids.view(-1, self.max_label_length),
                label_mask.view(-1, self.max_label_length),
            ).hidden_states[-1]
            hid_label = hid_label[:, 0, :]
            hid_label = hid_label.detach()
            self.value_lookup[s] = nn.Embedding.from_pretrained(hid_label, freeze=True)
            self.value_lookup[s].padding_idx = -1
            assert self.value_lookup[s].weight.shape == (label_id.size(0), self.bert_output_dim)

        print("Complete initialization of slot and value lookup")
        self.sv_encoder = None

    def forward(
        self,
        input_ids,
        token_type_ids,
        attention_mask,
        labels=None,
        n_gpu=1,
        target_slot=None,
    ):
        # B = Batch Size
        # M = Max Turn Size
        # N = Seq Len
        # J = Target_slot Len
        # H_GRU = RNN Hidden Dim
        
        # input_ids: [B, M, N]
        # token_type_ids: [B, M, N]
        # attention_mask: [B, M, N]
        # labels: [B, M, J]

        # if target_slot is not specified, output values corresponding all slot-types
        if target_slot is None:
            target_slot = list(range(0, self.num_slots))

        ds = input_ids.size(0)  # Batch size (B)
        ts = input_ids.size(1)  # Max turn size (M)
        bs = ds * ts # B * M
        slot_dim = len(target_slot)  # J

        # Utterance encoding
        hidden = self.utterance_encoder(
            input_ids.view(-1, self.max_seq_length),
            token_type_ids.view(-1, self.max_seq_length),
            attention_mask.view(-1, self.max_seq_length),
        ).hidden_states[-1] # [B*M, N, H]

        hidden = torch.mul(
            hidden,
            attention_mask.view(-1, self.max_seq_length, 1)
            .expand(hidden.size())
            .float(),
        )
        hidden = hidden.repeat(slot_dim, 1, 1)  # [J*M*B, N, H]

        hid_slot = self.slot_lookup.weight[
            target_slot, :
        ]  # Select target slot embedding # [J, H]
        hid_slot = hid_slot.repeat(1, bs).view(bs * slot_dim, -1)  # [J*M*B, H]

        # Attended utterance vector
        hidden = self.attn(
            hid_slot,  # q^s  [J*M*B, H] => [J*M*B, 1, H]
            hidden,  # U [J*M*B, N, H]
            hidden,  # U [J*M*B, N, H]
            mask=attention_mask.view(-1, 1, self.max_seq_length).repeat(slot_dim, 1, 1),
        ) # [J*M*B, 1, H] -> 1 = hid_slot_seq_len
        hidden = hidden.squeeze()  # h [J*M*B, H] Aggregated Slot Context
        hidden = hidden.view(slot_dim, ds, ts, -1).view(
            -1, ts, self.bert_output_dim
        )  # [J*B, M, H]

        # NBT
        if self.zero_init_rnn:
            h = torch.zeros(
                self.rnn_num_layers, input_ids.shape[0] * slot_dim, self.hidden_dim
            ).to(
                self.device
            )  # [1, slot_dim*ds, hidden]
        else:
            h = hidden[:, 0, :].unsqueeze(0).expand(self.rnn_num_layers, -1, -1) # 원래 repeat(self.rnn_num_layers, 1, 1) 이었는데 바꿈
            h = self.rnn_init_linear(h)    # 왜냐면 1 dim은 언제나 사이즈 1이니까 expand 사용하는게 더 좋음(expand 메모리 복사 없음, repeat 복사됨)

        if isinstance(self.nbt, nn.GRU):
            rnn_out, _ = self.nbt(hidden, h)  # [J*B, M, H_GRU]
        elif isinstance(self.nbt, nn.LSTM):
            c = torch.zeros(
                self.rnn_num_layers, input_ids.shape[0] * slot_dim, self.hidden_dim
            ).to(
                self.device
            )  # [1, slot_dim*ds, hidden]
            rnn_out, _ = self.nbt(hidden, (h, c))  # [slot_dim*ds, turn, hidden]
        rnn_out = self.layer_norm(self.linear(self.dropout(rnn_out)))

        hidden = rnn_out.view(slot_dim, ds, ts, -1)  # [J, B, M, H]  변경 전 # [J, B, M, H_G]

        # Label (slot-value) encoding
        loss = 0
        loss_slot = []
        pred_slot = []
        output = []
        for s, slot_id in enumerate(target_slot):  ## note: target_slots are successive
            # loss calculation
            hid_label = self.value_lookup[slot_id].weight
            num_slot_labels = hid_label.size(0)

            _hid_label = (
                hid_label.unsqueeze(0)
                .unsqueeze(0)
               .repeat(ds, ts, 1, 1)
                .view(ds * ts * num_slot_labels, -1)
            )
            _hidden = (
                hidden[s, :, :, :]
                .unsqueeze(2)
                .repeat(1, 1, num_slot_labels, 1)
                .view(ds * ts * num_slot_labels, -1)
            )
            _dist = self.metric(_hid_label, _hidden).view(ds, ts, num_slot_labels)
            _dist = -_dist
            _, pred = torch.max(_dist, -1)
            pred_slot.append(pred.view(ds, ts, 1))
            output.append(_dist)

            if labels is not None:
                _loss = self.nll(_dist.view(ds * ts, -1), labels[:, :, s].view(-1))
                loss_slot.append(_loss.item())
                loss += _loss

        pred_slot = torch.cat(pred_slot, 2)
        
        if labels is None:
            return output, pred_slot

        # calculate joint accuracy
        accuracy = (pred_slot == labels).view(-1, slot_dim)
        acc_slot = (
            torch.sum(accuracy, 0).float()
            / torch.sum(labels.view(-1, slot_dim) > -1, 0).float()
        )
        acc = (
            sum(torch.sum(accuracy, 1) / slot_dim).float()
            / torch.sum(labels[:, :, 0].view(-1) > -1, 0).float()
        )  # joint accuracy
        
#         acc2 = ( # 별 차이 없으니까 그냥 기존거 쓰자
#             torch.sum(accuracy).float()
#             / torch.sum(labels > -1).float()
#         )  # joint accuracy
#         assert acc == acc2, f'acc: {acc}  acc2: {acc2}'

        if n_gpu == 1:
            return loss, loss_slot, acc, acc_slot, pred_slot
        else:
            return (
                loss.unsqueeze(0),
                None,
                acc.unsqueeze(0),
                acc_slot.unsqueeze(0),
                pred_slot.unsqueeze(0),
            )

    @staticmethod
    def init_parameter(module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_normal_(module.weight)
            torch.nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.GRU) or isinstance(module, nn.LSTM):
            torch.nn.init.xavier_normal_(module.weight_ih_l0)
            torch.nn.init.xavier_normal_(module.weight_hh_l0)
            torch.nn.init.constant_(module.bias_ih_l0, 0.0)
            torch.nn.init.constant_(module.bias_hh_l0, 0.0)

In [23]:
import torch.nn as nn
import torch.nn.functional as F

def tokenize_ontology(ontology, tokenizer, max_seq_length):
    slot_types = []
    slot_values = []
    for k, v in ontology.items():
        tokens = tokenizer.encode(k)
        if len(tokens) < max_seq_length:
            gap = max_seq_length - len(tokens)
            tokens.extend([tokenizer.pad_token_id] * gap)
        slot_types.append(tokens)
        slot_value = []
        for vv in v:
            tokens = tokenizer.encode(vv)
            if len(tokens) < max_seq_length:
                gap = max_seq_length - len(tokens)
                tokens.extend([tokenizer.pad_token_id] * gap)
            slot_value.append(tokens)
        slot_values.append(torch.LongTensor(slot_value))
    return torch.LongTensor(slot_types), slot_values

In [24]:
slot_type_ids, slot_values_ids = tokenize_ontology(ontology, tokenizer, 12)
num_labels = [len(s) for s in slot_values_ids]

print('Tokenized Slot: ', slot_type_ids.size())
cnt = 0
for slot, slot_value_id in zip(slot_meta, slot_values_ids):
    print(f'Tokenized Value of {slot}', slot_value_id.size())
    cnt += 1
    if cnt > 4:
        break

Tokenized Slot:  torch.Size([45, 12])
Tokenized Value of 관광-경치 좋은 torch.Size([4, 12])
Tokenized Value of 관광-교육적 torch.Size([4, 12])
Tokenized Value of 관광-도보 가능 torch.Size([4, 12])
Tokenized Value of 관광-문화 예술 torch.Size([4, 12])
Tokenized Value of 관광-역사적 torch.Size([4, 12])


In [25]:
num_labels = [len(s) for s in slot_values_ids]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = 1 if torch.cuda.device_count() < 2 else torch.cuda.device_count()
n_epochs = args.num_train_epochs

In [26]:
model = SUMBT(args, num_labels, device)
model.initialize_slot_value_lookup(slot_values_ids, slot_type_ids)  # Tokenized Ontology의 Pre-encoding using BERT_SV
model.to(device)
print(device)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Complete initialization of slot and value lookup
cuda


In [27]:
model.slot_lookup.weight.size()

torch.Size([45, 768])

In [28]:
slot_type_ids.shape

torch.Size([45, 12])

In [29]:
from data_utils import WOSDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import random


train_data = WOSDataset(train_features)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=8, sampler=train_sampler, collate_fn=processor.collate_fn)

dev_data = WOSDataset(dev_features)
dev_sampler = SequentialSampler(dev_data)
dev_loader = DataLoader(dev_data, batch_size=8, sampler=dev_sampler, collate_fn=processor.collate_fn)

In [30]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

t_total = len(train_loader) * n_epochs
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(t_total * args.warmup_ratio), num_training_steps=t_total
)

In [31]:
from evaluation import _evaluation

In [32]:
def inference(model, eval_loader, processor, device):
    model.eval()
    predictions = {}
    
    for step, batch in tqdm(enumerate(eval_loader), total=len(eval_loader)):
        input_ids, segment_ids, input_masks, target_ids, num_turns, guids  = \
            [b.to(device) if not isinstance(b, list) else b for b in batch]
        
        with torch.no_grad():
            with autocast(enabled=use_amp):
                output, pred_slot = model(input_ids, segment_ids, input_masks, None, 1)
            
        pred_slot = pred_slot.detach().cpu()
        
        for guid, num_turn, p_slot in zip(guids, num_turns, pred_slot):
            pred_states = processor.recover_state(p_slot, num_turn)
            for t_idx, pred_state in enumerate(pred_states):
                predictions[f'{guid}-{t_idx}'] = pred_state
    
    return predictions

In [33]:

scaler = GradScaler(enabled=use_amp)

best_score, best_checkpoint = 0, 0
for epoch in tqdm(range(n_epochs)):
    batch_loss = []
    model.train()
    for step, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        
        input_ids, segment_ids, input_masks, target_ids, num_turns, guids  = \
        [b.to(device) if not isinstance(b, list) else b for b in batch]

        # Forward
        with autocast(enabled=use_amp):
            if n_gpu == 1:
                loss, loss_slot, acc, acc_slot, _ = model(input_ids, segment_ids, input_masks, target_ids, n_gpu)
            else:
                loss, _, acc, acc_slot, _ = model(input_ids, segment_ids, input_masks, target_ids, n_gpu)
        
        batch_loss.append(loss.item())

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        
        scale = scaler.get_scale()
        scaler.update()
        step_scheduler = scaler.get_scale() == scale
        
        if step_scheduler:
            scheduler.step()


        if step % 100 == 0:
            print('[%d/%d] [%d/%d] %f' % (epoch, n_epochs, step, len(train_loader), loss.item()))
            
    predictions = inference(model, dev_loader, processor, device)
    eval_result = _evaluation(predictions, dev_labels, slot_meta)
    for k, v in eval_result.items():
        print(f"{k}: {v}")

[0/10] [0/788] 110.114372
[0/10] [100/788] 42.105022
[0/10] [200/788] 40.169041
[0/10] [300/788] 31.595776
[0/10] [400/788] 32.986732
[0/10] [500/788] 30.214699
[0/10] [600/788] 23.632011
[0/10] [700/788] 23.125034




{'joint_goal_accuracy': 0.024, 'turn_slot_accuracy': 0.8592622222222255, 'turn_slot_f1': 0.25224322985130887}
joint_goal_accuracy: 0.024
turn_slot_accuracy: 0.8592622222222255
turn_slot_f1: 0.25224322985130887


[1/10] [0/788] 17.328161
[1/10] [100/788] 20.273682
[1/10] [200/788] 16.353893
[1/10] [300/788] 17.911474
[1/10] [400/788] 14.494902
[1/10] [500/788] 17.318621
[1/10] [600/788] 15.776504
[1/10] [700/788] 14.748581




{'joint_goal_accuracy': 0.2174, 'turn_slot_accuracy': 0.9455111111111172, 'turn_slot_f1': 0.753052711805542}
joint_goal_accuracy: 0.2174
turn_slot_accuracy: 0.9455111111111172
turn_slot_f1: 0.753052711805542


[2/10] [0/788] 8.080420
[2/10] [100/788] 12.932704
[2/10] [200/788] 9.851366
[2/10] [300/788] 6.188788
[2/10] [400/788] 5.190873
[2/10] [500/788] 8.912785
[2/10] [600/788] 7.961097
[2/10] [700/788] 7.608935



...


{'joint_goal_accuracy': 0.7136, 'turn_slot_accuracy': 0.9899422222222276, 'turn_slot_f1': 0.9633212478024045}
joint_goal_accuracy: 0.7136
turn_slot_accuracy: 0.9899422222222276
turn_slot_f1: 0.9633212478024045


[8/10] [0/788] 1.431324
[8/10] [100/788] 1.548443
[8/10] [200/788] 1.633593
[8/10] [300/788] 0.862671
[8/10] [400/788] 1.790905
[8/10] [500/788] 1.428502
[8/10] [600/788] 2.430533
[8/10] [700/788] 2.556275




{'joint_goal_accuracy': 0.7278, 'turn_slot_accuracy': 0.9904177777777817, 'turn_slot_f1': 0.9651944989870144}
joint_goal_accuracy: 0.7278
turn_slot_accuracy: 0.9904177777777817
turn_slot_f1: 0.9651944989870144


[9/10] [0/788] 0.735247
[9/10] [100/788] 2.006325
[9/10] [200/788] 2.043746
[9/10] [300/788] 1.890898
[9/10] [400/788] 0.950637
[9/10] [500/788] 1.352677
[9/10] [600/788] 2.410396
[9/10] [700/788] 2.371645




{'joint_goal_accuracy': 0.7312, 'turn_slot_accuracy': 0.9906177777777823, 'turn_slot_f1': 0.9658299304072602}
joint_goal_accuracy: 0.7312
turn_slot_accuracy: 0.9906177777777823
turn_slot_f1: 0.9658299304072602



In [34]:
eval_data = json.load(open(f"/opt/ml/input/data/eval_dataset/eval_dials.json", "r"))

eval_examples = get_examples_from_dialogues(
    eval_data, user_first=True, dialogue_level=True
)

# Extracting Featrues
eval_features = processor.convert_examples_to_features(eval_examples)
eval_data = WOSDataset(eval_features)
eval_sampler = SequentialSampler(eval_data)
eval_loader = DataLoader(
    eval_data,
    batch_size=8,
    sampler=eval_sampler,
    collate_fn=processor.collate_fn,
)

In [35]:
predictions = inference(model, eval_loader, processor, device)

In [36]:
json.dump(predictions, open('predictions/koelectra_base.csv', 'w'), indent=2, ensure_ascii=False) 